In [1]:
import os, sys
import warnings
import timeit
import numpy as np
import matplotlib.pyplot as plt
import re
import pandas as pd

from collections import defaultdict

In [2]:
data_a=pd.read_excel('../postprocessed-data/dbem3_a_pergene_insertions.xlsx',engine='openpyxl')
data_a=data_a.drop(['Unnamed: 0'],axis=1)

data_b=pd.read_excel('../postprocessed-data/dbem3_b_pergene_insertions.xlsx',engine='openpyxl')
data_b=data_b.drop(['Unnamed: 0'],axis=1)
# data.fillna(1,inplace=True)
data_merged=pd.read_excel('../postprocessed-data/dbem3_merged_pergene_insertions.xlsx',engine='openpyxl')
data_merged=data_merged.drop(['Unnamed: 0'],axis=1)

In [5]:
data_a.head(2)

,Gene name,Chromosome,Start location,End location,Insertion locations,Reads per insertion location,Reads,Insertions
0,YAL069W,I,335,649,"[337, 337, 341, 344, 350, 352, 368, 370, 387, ...","[2, 93, 12, 57, 25, 1, 4, 2, 120, 16, 1, 12, 1...",884,40
1,YAL068W-A,I,538,792,"[567, 571, 572, 578, 579, 589, 595, 687, 691, ...","[71, 5, 1, 1, 140, 8, 31, 137, 214, 19, 3, 1, ...",428,18


In [22]:
data_b.head(2)

,Gene name,Chromosome,Start location,End location,Insertion locations,Reads per insertion location,Reads,Insertions
0,YAL069W,I,335,649,"[336, 340, 349, 372, 377, 382, 386, 386, 387, ...","[3, 327, 87, 1, 3, 12, 39, 1, 43, 6, 3, 45, 1,...",2353,55
1,YAL068W-A,I,538,792,"[559, 561, 564, 565, 569, 570, 570, 571, 573, ...","[6, 1, 2, 2, 147, 9, 154, 17, 177, 1, 132, 5, ...",694,25


In [23]:
data_merged.head(2)

,Gene name,Chromosome,Start location,End location,Insertion locations,Reads per insertion location,Reads,Insertions
0,YAL069W,I,335,649,"[336, 337, 340, 344, 349, 352, 368, 371, 377, ...","[3, 2, 430, 57, 114, 1, 4, 3, 3, 12, 55, 1, 14...",3428,67
1,YAL068W-A,I,538,792,"[559, 561, 564, 568, 570, 570, 571, 573, 576, ...","[6, 3, 2, 202, 9, 154, 22, 161, 2, 1, 296, 12,...",1207,31


In [3]:
from from_excel_to_list import from_excel_to_list

In [4]:
#functional version of recursive flatten which handles both tuples and lists, and lets you throw in any mix of positional arguments. 
# Returns a generator which produces the entire sequence in order,
flatten = lambda *n: (e for a in n
    for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))

In [4]:
def from_replicates_to_merged(replicates):
    """
    Takes a list of replicates and returns a single merged dataset.
  
    """
    #functional version of recursive flatten which handles both tuples and lists, and lets you throw in any mix of positional arguments. 
    # Returns a generator which produces the entire sequence in order,
    flatten = lambda *n: (e for a in n
        for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))
        
    # First, flatten the replicates
    replicates = list(flatten(replicates))

    data_merged_calculation=replicates[0].copy() # make a copy of the first replicate 
    data_merged_calculation.loc[:,"merged_reads"]=np.zeros(len(data_merged_calculation))
    data_merged_calculation.loc[:,"merged_insertions"]=np.zeros(len(data_merged_calculation))
    a=[]
    a_reads=[]
    for i in np.arange(0,len(replicates[0])):
        
        a.append(from_excel_to_list(replicates[0]['Insertion locations'][i]))
        b=from_excel_to_list(replicates[1]['Insertion locations'][i])
        a_reads.append(from_excel_to_list(replicates[0]['Reads per insertion location'][i]))
        b_reads=from_excel_to_list(replicates[1]['Reads per insertion location'][i])

        union=list(flatten(a,b)) # concatenate all the insertions per location for each replicate
        unique_union,unique_index,unique_counts=np.unique(union,return_index=True,return_counts=True)

        union_reads=list(flatten(a_reads,b_reads)) # concatenate all the reads per insertion location of all replicates
            
        union_dict=dict(zip(unique_union,unique_counts)) # make a dictionary of the union

        union_dict_pd=pd.DataFrame.from_dict(union_dict,orient='index',columns=['count']) # make a dataframe of the union

        union_dict_pd['insertions']=union_dict_pd.index
        union_dict_pd.index=np.arange(0,len(union_dict_pd))

        union_dict_pd['reads']=np.zeros(len(union_dict_pd))
        union_dict_pd['merged_reads']=np.zeros(len(union_dict_pd))

        for j in np.arange(0,len(union_dict_pd)):
            # if type(union_reads)!=int:
            union_dict_pd.loc[:,'reads'][j]=union_reads[j]
            union_dict_pd.loc[:,'merged_reads'][j]=union_dict_pd.loc[:,'count'][j]*union_dict_pd.loc[:,'reads'][j]

        total_reads=np.sum(union_dict_pd['merged_reads'])-np.max(union_dict_pd['merged_reads'])

        data_merged_calculation.loc[:,'Insertion locations'][i]=union_dict_pd['insertions'].values
        data_merged_calculation.loc[:,'merged_reads'][i]=total_reads
        data_merged_calculation.loc[:,'Reads per insertion location'][i]=union_dict_pd['reads'].values
        data_merged_calculation.loc[:,'merged_insertions'][i]=len(union_dict_pd['insertions'].values)

    return data_merged_calculation

In [5]:
data_merged=from_replicates_to_merged([data_a,data_b])

/tmp/ipykernel_8016/2447102373.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_dict_pd.loc[:,'reads'][j]=union_reads[j]
/tmp/ipykernel_8016/2447102373.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_dict_pd.loc[:,'merged_reads'][j]=union_dict_pd.loc[:,'count'][j]*union_dict_pd.loc[:,'reads'][j]
/tmp/ipykernel_8016/2447102373.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merged_calculation.loc[:,'Insert

In [72]:
data_merged_calculation=data_merged.copy()
data_merged_calculation.loc[:,"merged_reads"]=np.zeros(len(data_merged_calculation))
data_merged_calculation.loc[:,"merged_insertions"]=np.zeros(len(data_merged_calculation))

for i in np.arange(0,len(data_a)):
    a=from_excel_to_list(data_a['Insertion locations'][i])
    b=from_excel_to_list(data_b['Insertion locations'][i])
    a_reads=from_excel_to_list(data_a['Reads per insertion location'][i])
    b_reads=from_excel_to_list(data_b['Reads per insertion location'][i])

    if type(a)!=np.int64 and type(b)!=np.int64 and type(a)!=int and type(b)!=int:
        union=a+b
        unique_union,unique_index,unique_counts=np.unique(union,return_index=True,return_counts=True)
        
    else :
        union=list(flatten(a,b))
        unique_union,unique_index,unique_counts=np.unique(union,return_index=True,return_counts=True)

    if type(a_reads)!=np.int64 and type(b_reads)!=np.int64 and type(a_reads)!=int and type(b_reads)!=int:
        union_reads=a_reads+b_reads
    
    else :
        union_reads=list(flatten(a_reads,b_reads))
        
    union_dict=dict(zip(unique_union,unique_counts))

    union_dict_pd=pd.DataFrame.from_dict(union_dict,orient='index',columns=['count'])

    union_dict_pd['insertions']=union_dict_pd.index
    union_dict_pd.index=np.arange(0,len(union_dict_pd))

    union_dict_pd['reads']=np.zeros(len(union_dict_pd))
    union_dict_pd['merged_reads']=np.zeros(len(union_dict_pd))

    for j in np.arange(0,len(union_dict_pd)):
        if type(union_reads)!=int:
            union_dict_pd.loc[:,'reads'][j]=union_reads[j]
            union_dict_pd.loc[:,'merged_reads'][j]=union_dict_pd.loc[:,'count'][j]*union_dict_pd.loc[:,'reads'][j]

    total_reads=np.sum(union_dict_pd['merged_reads'])-np.max(union_dict_pd['merged_reads'])

    data_merged_calculation.loc[:,'Insertion locations'][i]=union_dict_pd['insertions'].values
    data_merged_calculation.loc[:,'merged_reads'][i]=total_reads
    data_merged_calculation.loc[:,'Reads per insertion location'][i]=union_dict_pd['reads'].values
    data_merged_calculation.loc[:,'merged_insertions'][i]=len(union_dict_pd['insertions'].values)


/tmp/ipykernel_7611/2153808316.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_dict_pd.loc[:,'reads'][j]=union_reads[j]
/tmp/ipykernel_7611/2153808316.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_dict_pd.loc[:,'merged_reads'][j]=union_dict_pd.loc[:,'count'][j]*union_dict_pd.loc[:,'reads'][j]
/tmp/ipykernel_7611/2153808316.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merged_calculation.loc[:,'Insert

In [73]:
data_merged_calculation

,Gene name,Chromosome,Start location,End location,Insertion locations,Reads per insertion location,Reads,Insertions,merged_reads
0,YAL069W,I,335,649,"[336.0, 337.0, 340.0, 341.0, 344.0, 349.0, 350...","[2.0, 93.0, 12.0, 57.0, 25.0, 1.0, 4.0, 2.0, 1...",3428,78,3300.0
1,YAL068W-A,I,538,792,"[559.0, 561.0, 564.0, 565.0, 567.0, 569.0, 570...","[71.0, 5.0, 1.0, 1.0, 140.0, 8.0, 31.0, 137.0,...",1207,40,1536.0
2,PAU8,I,1807,2169,"[1810.0, 1819.0, 1825.0, 1826.0, 1831.0, 1833....","[16.0, 11.0, 35.0, 1.0, 9.0, 4.0, 2.0, 1.0, 71...",663,48,577.0
3,YAL067W-A,I,2480,2707,"[2494.0, 2528.0, 2534.0, 2552.0, 2560.0, 2561....","[75.0, 1.0, 2.0, 1.0, 2.0, 11.0, 36.0, 19.0, 1...",353,27,310.0
4,SEO1,I,7235,9016,"[7238.0, 7243.0, 7248.0, 7293.0, 7294.0, 7298....","[1.0, 30.0, 1.0, 40.0, 5.0, 1.0, 1.0, 2.0, 1.0...",2776,132,2504.0
...,...,...,...,...,...,...,...,...,...
6595,ARR3,XVI,939922,941136,"[939930.0, 939931.0, 939933.0, 939934.0, 93993...","[21.0, 1.0, 1.0, 8.0, 1.0, 16.0, 25.0, 27.0, 1...",1976,136,1796.0
6596,YPR202W,XVI,943032,943896,"[943035.0, 943041.0, 943062.0, 943063.0, 94307...","[28.0, 2.0, 6.0, 5.0, 2.0, 1.0, 13.0, 26.0, 1....",1527,155,1413.0
6597,YPR203W,XVI,943880,944188,"[943880.0, 943883.0, 943885.0, 943887.0, 94389...","[11.0, 3.0, 27.0, 1.0, 22.0, 2.0, 5.0, 10.0, 3...",1220,104,1018.0
6598,YPR204W,XVI,944603,947701,"[944607.0, 944615.0, 944623.0, 944624.0, 94462...","[13.0, 1.0, 3.0, 2.0, 3.0, 5.0, 8.0, 1.0, 1.0,...",6863,692,6523.0


In [8]:
a=from_excel_to_list(data_a['Insertion locations'][0])
b=from_excel_to_list(data_b['Insertion locations'][0])
a_reads=from_excel_to_list(data_a['Reads per insertion location'][0])
b_reads=from_excel_to_list(data_b['Reads per insertion location'][0])

In [45]:
union=a+b
unique_union,unique_index,unique_counts=np.unique(union,return_index=True,return_counts=True)

union_reads=a_reads+b_reads

union_dict=dict(zip(unique_union,unique_counts))

union_dict_pd=pd.DataFrame.from_dict(union_dict,orient='index',columns=['count'])

union_dict_pd['insertions']=union_dict_pd.index
union_dict_pd.index=np.arange(0,len(union_dict_pd))

union_dict_pd['reads']=np.zeros(len(union_dict_pd))
union_dict_pd['merged_reads']=np.zeros(len(union_dict_pd))

for i in range(len(union_dict_pd)):
    union_dict_pd.loc[:,'reads'][i]=union_reads[i]
    union_dict_pd.loc[:,'merged_reads'][i]=union_dict_pd.loc[:,'count'][i]*union_dict_pd.loc[:,'reads'][i]

np.sum(union_dict_pd['merged_reads'])-np.max(union_dict_pd['merged_reads'])

/tmp/ipykernel_6031/2996342652.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_dict_pd.loc[:,'reads'][i]=union_reads[i]
/tmp/ipykernel_6031/2996342652.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_dict_pd.loc[:,'merged_reads'][i]=union_dict_pd.loc[:,'count'][i]*union_dict_pd.loc[:,'reads'][i]


3300.0

In [132]:
set_a=set(a)
set_b=set(b)
s = set_a.union(set_b)  # union of both sets

# Difference in two sets
diff_element_a = set_b - set_a
diff_element_b = set_a - set_b
  
# union of difference + first list
output = a + list(diff_element_a) + list(diff_element_b)



In [133]:
d = defaultdict(int)
for k in output:
    d[k] += 1
    

In [134]:
merged_insertions=pd.DataFrame(d.items(), columns=['Insertion locations', 'Count'])
merged_insertions.sort_values(by='Count', inplace=True,ascending=False)

In [109]:
a_reads_dict=defaultdict(int)
j=0
for k in a:
    reads=a_reads[j]
    a_reads_dict[k] = reads
    j=j+1



In [142]:
def get_reads_with_insertions(replicate_reads_per_gene,replicate_insertions_per_gene):
    """Get the reads per insertion location for a given replicate

    Parameters
    ----------
    replicate_reads_per_gene : list
        output from from_excel_to_list function
    """
    reads_dict=defaultdict(int)
    j=0
    for k in replicate_insertions_per_gene:
        reads=replicate_reads_per_gene[j]
        reads_dict[k] = reads
        j=j+1
    return reads_dict

In [143]:
a_reads_dict=get_reads_with_insertions(a_reads,a)
b_reads_dict=get_reads_with_insertions(b_reads,b)

In [144]:
a_reads_df=pd.DataFrame(a_reads_dict.items(), columns=['Insertion locations', 'Reads per insertion location'])
b_reads_df=pd.DataFrame(b_reads_dict.items(), columns=['Insertion locations', 'Reads per insertion location'])

In [164]:
a_reads_merged=pd.merge(pd.merge(merged_insertions,a_reads_df,on='Insertion locations'),b_reads_df,on='Insertion locations',how='left')
a_reads_merged.fillna(0,inplace=True)

In [175]:
print(np.sum(b_reads_df['Reads per insertion location'])-np.max(b_reads_df["Reads per insertion location"]))
print(np.sum(a_reads_df['Reads per insertion location'])-np.max(a_reads_df["Reads per insertion location"]))

2249.0
882.0


In [168]:
print(np.sum(a_reads_merged['Reads per insertion location_x']))
print(np.sum(a_reads_merged['Reads per insertion location_y']))

1096.0
239.0


In [156]:
group=a_reads_merged.groupby("Count").sum()
group

,Insertion locations,Reads per insertion location_x,Reads per insertion location_y
Count,,,
1,6110.0,192.0,239.0
2,11623.0,811.0,0.0
3,337.0,93.0,0.0


In [158]:
total=[]
for i in group.index:
    print(i)
    total.append(group.loc[i]['Reads per insertion location_x']*i+group.loc[i]['Reads per insertion location_y']*i)

1
2
3


In [160]:
np.sum(total)

2332.0